In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


讀取的檔案，已經涵蓋了新聞的正負面屬性，今天我們希望找出每篇新聞的關鍵字  
因此除了要做結巴切詞之外，同時也希望得到tf-idf等等資訊  
不過再這之前，我們希望對新聞資訊做一些處理(處理亂碼、標點符號、停用詞等等)  
更進階一點，為了讓結巴切的準確一些，我們可以新增字典，讓結巴沒看過的詞彙也分的出來   

In [3]:
LARGAN_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/大立光正負新聞.csv')
Foxconn_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/鴻海正負新聞.csv')
TSMC_NEWS = pd.read_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/台積電正負新聞.csv')


### 正則表達式把文章弄得好看一點

## 切割出重要詞彙

In [198]:
def cut(df):

    upnews = df[df['status']==1]
    downnews = df[df['status']==-1]
    
    positive_token = []
    for goodnews in upnews['CONTENT']:
        goodnews = re.sub(r'[。，（）]','',goodnews)
        #這邊想再加一行過濾亂碼的條件，但是還沒搞懂
        seg_list = jieba.cut_for_search(news)
        token = ' '.join(seg_list)
        positive_token.append(token)
    
    count_vec=CountVectorizer() 
    good_news_count = count_vec.fit_transform(positive_token) 
    good_news_count = pd.DataFrame(good_news_count.toarray(),columns = count_vec.get_feature_names())
    #這兩行如果不做的話，concat起來會因為index不同而出現NaN
    good_news_count.reset_index(drop=True, inplace=True)  
    upnews.reset_index(drop=True, inplace=True)

    positive_news = pd.concat([upnews,good_news_count],axis = 1)

    
    negative_token = []
    for badnews in downnews['CONTENT']:
        badnews = re.sub(r'[。，（）]','',badnews)
        seg_list = jieba.cut_for_search(badnews)
        token = ' '.join(seg_list)
        negative_token.append(token)
    
    count_vec=CountVectorizer() 
    bad_news_count = count_vec.fit_transform(negative_token) 
    bad_news_count = pd.DataFrame(bad_news_count.toarray(),columns = count_vec.get_feature_names())
    
    bad_news_count.reset_index(drop=True, inplace=True)  
    downnews.reset_index(drop=True, inplace=True)
    
    
    negative_news = pd.concat([downnews,bad_news_count],axis = 1)
    
    return positive_news,negative_news

In [199]:
positive_Foxconn_NEWS = cut(Foxconn_NEWS)[0]
negative_Foxconn_NEWS = cut(Foxconn_NEWS)[1]
positive_LARGAN_NEWS = cut(LARGAN_NEWS)[0]
negative_LARGAN_NEWS = cut(LARGAN_NEWS)[1]
positive_TSMC_NEWS = cut(TSMC_NEWS)[0]
negative_TSMC_NEWS = cut(TSMC_NEWS)[1]

In [209]:
positive_Foxconn_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/鴻海正面新聞.csv')
negative_Foxconn_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/鴻海負面新聞.csv')
positive_LARGAN_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/大立光正面新聞.csv')
negative_LARGAN_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/大立光負面新聞.csv')
positive_TSMC_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/台積電正面新聞.csv')
negative_TSMC_NEWS.to_csv('/Users/liaoshousan/Desktop/大四上/RLSD 期末專案/台積電負面新聞.csv')